# [Go to "Styling" in pandas docs](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html)

In [1]:
import pandas as pd
import numpy as np

The `DataFrame.style` property returns a `Styler` object, which has useful methods for formatting and displaying `DataFrames`.

Styling is accomplished using **CSS**: through functions that take scalars, `DataFrames` or `Series`, and return like-indexed `DataFrames` or `Series` with CSS `"attribute: value"` pairs for the values.

>`Styler` objects have a `_repr_html_` method that automatically renders tables.

In [2]:
df = pd.DataFrame(np.random.standard_normal(20).reshape(5, 4), columns=list('ABCD'))
print(type(df.style))

df.style

<class 'pandas.io.formats.style.Styler'>


,A,B,C,D
0,1.565390,0.799185,-2.579131,0.505232
1,0.293497,1.412492,1.745990,-0.005426
2,1.005967,0.444021,0.062143,-0.416147
3,-0.326980,-0.110420,-2.160929,-0.425460
4,-0.952315,-1.272838,-0.758581,0.426546


>If you want the actual HTML back for further processing or for writing to file, call the `.render()` method, which returns a `string` with CSS classes for each cell.

In [3]:
df[:2].style.render()

'<style  type="text/css" >\n</style><table id="T_94f8e_" ><thead>    <tr>        <th class="blank level0" ></th>        <th class="col_heading level0 col0" >A</th>        <th class="col_heading level0 col1" >B</th>        <th class="col_heading level0 col2" >C</th>        <th class="col_heading level0 col3" >D</th>    </tr></thead><tbody>\n                <tr>\n                        <th id="T_94f8e_level0_row0" class="row_heading level0 row0" >0</th>\n                        <td id="T_94f8e_row0_col0" class="data row0 col0" >1.565390</td>\n                        <td id="T_94f8e_row0_col1" class="data row0 col1" >0.799185</td>\n                        <td id="T_94f8e_row0_col2" class="data row0 col2" >-2.579131</td>\n                        <td id="T_94f8e_row0_col3" class="data row0 col3" >0.505232</td>\n            </tr>\n            <tr>\n                        <th id="T_94f8e_level0_row1" class="row_heading level0 row1" >1</th>\n                        <td id="T_94f8e_row1_col0"

# 1. Building Styles

Style functions should return strings with one or more CSS `attribute: value` pairs delimited by semicolons. Use:

- `Styler.applymap(func)` for elementwise styles

- `Styler.apply(func, axis=0)` for columnwise styles

- `Styler.apply(func, axis=1)` for rowwise styles

- `Styler.apply(func, axis=None)` for tablewise styles

## 1.1 `Styler.applymap` - elementwise.

In [4]:
def color_negatives_red(val):
    """ Converts -ve values' color to red."""
    
    color = 'red' if val < 0 else 'black'
    return f'color: {color}'


df.style.applymap(color_negatives_red)

,A,B,C,D
0,1.565390,0.799185,-2.579131,0.505232
1,0.293497,1.412492,1.745990,-0.005426
2,1.005967,0.444021,0.062143,-0.416147
3,-0.326980,-0.110420,-2.160929,-0.425460
4,-0.952315,-1.272838,-0.758581,0.426546


## 1.2 `Styler.apply` - column-wise(axis=0)

In [5]:
def highlight_max(s):
    """
    Highlight the maximum value in a column (axis=0) or row(axis=1) yellow
    """
    max_bool_array = (s == s.max())
    return ['background-color: yellow' if is_max else '' 
            for is_max in max_bool_array ]

df.style.apply(highlight_max, axis=0)

,A,B,C,D
0,1.565390,0.799185,-2.579131,0.505232
1,0.293497,1.412492,1.745990,-0.005426
2,1.005967,0.444021,0.062143,-0.416147
3,-0.326980,-0.110420,-2.160929,-0.425460
4,-0.952315,-1.272838,-0.758581,0.426546


## 1.3 `Styler.apply` -  row-wise(axis=1)

In [6]:
df.style.apply(highlight_max, axis=1)

,A,B,C,D
0,1.565390,0.799185,-2.579131,0.505232
1,0.293497,1.412492,1.745990,-0.005426
2,1.005967,0.444021,0.062143,-0.416147
3,-0.326980,-0.110420,-2.160929,-0.425460
4,-0.952315,-1.272838,-0.758581,0.426546


## 1.4 `Styler.apply` - table-wise(axis=None)

>The function must return a `DataFrame` with the same index and column labels.

In [7]:
def enlarge_overall_max(data):
    """
    Highlight the maximum in a Series or DataFrame colums/rows yellow
    """
    return pd.DataFrame(np.where(data == data.max().max(), 'font-size: 2em', ''),
                        index=data.index, columns=data.columns)
   
df.style.apply(enlarge_overall_max, axis=None)  

,A,B,C,D
0,1.565390,0.799185,-2.579131,0.505232
1,0.293497,1.412492,1.745990,-0.005426
2,1.005967,0.444021,0.062143,-0.416147
3,-0.326980,-0.110420,-2.160929,-0.425460
4,-0.952315,-1.272838,-0.758581,0.426546


# 2. Slicing (specifying rows/columns)

Both `Styler.apply` and `Styler.applymap` accept a `subset` keyword, which allows you to apply styles to specific rows or columns:

In [8]:
df.style.apply(highlight_max, subset=['B', 'C', 'D'], axis=0)\
    .apply(enlarge_overall_max, axis=None)\
    .applymap(color_negatives_red, subset=['A', 'C'])

,A,B,C,D
0,1.565390,0.799185,-2.579131,0.505232
1,0.293497,1.412492,1.745990,-0.005426
2,1.005967,0.444021,0.062143,-0.416147
3,-0.326980,-0.110420,-2.160929,-0.425460
4,-0.952315,-1.272838,-0.758581,0.426546


To select a subset both column-wise and row-wise, use `pd.IndexSlice`. Only label-based slicing is currently supported. 

In [9]:
df.style.applymap(color_negatives_red, subset=pd.IndexSlice[0:2, ['A', 'C']])

,A,B,C,D
0,1.565390,0.799185,-2.579131,0.505232
1,0.293497,1.412492,1.745990,-0.005426
2,1.005967,0.444021,0.062143,-0.416147
3,-0.326980,-0.110420,-2.160929,-0.425460
4,-0.952315,-1.272838,-0.758581,0.426546


# 3. Formating Display Values

The display value can be controlled using `Styler.format`.

Cells can be formatted according to a [format spec string][1] or a callable that takes a single value and returns a string.

[1]: https://docs.python.org/3/library/string.html#format-specification-mini-language

In [10]:
df.style.format("{:.2%}")

,A,B,C,D
0,156.54%,79.92%,-257.91%,50.52%
1,29.35%,141.25%,174.60%,-0.54%
2,100.60%,44.40%,6.21%,-41.61%
3,-32.70%,-11.04%,-216.09%,-42.55%
4,-95.23%,-127.28%,-75.86%,42.65%


> You can use a `dict` to specify columns

In [11]:
df.style.format({'A': '{:e}', 'B': '{:.3f}', 'D': '{:%}'})

,A,B,C,D
0,1.565390e+00,0.799,-2.579131,50.523173%
1,2.934972e-01,1.412,1.745990,-0.542552%
2,1.005967e+00,0.444,0.062143,-41.614669%
3,-3.269803e-01,-0.110,-2.160929,-42.545965%
4,-9.523149e-01,-1.273,-0.758581,42.654583%


> Pass a callable (or dictionary of callables) for more flexible handling

In [12]:
df.style.format({"C": lambda x: f"±{abs(x):.2f}"})

,A,B,C,D
0,1.565390,0.799185,±2.58,0.505232
1,0.293497,1.412492,±1.75,-0.005426
2,1.005967,0.444021,±0.06,-0.416147
3,-0.326980,-0.110420,±2.16,-0.425460
4,-0.952315,-1.272838,±0.76,0.426546


>You can format the text displayed for missing values using `na_rep`

In [13]:
df.iloc[[0, 3], 1:3] = np.nan
df.style.format("{:.2%}", na_rep="-")

,A,B,C,D
0,156.54%,-,-,50.52%
1,29.35%,141.25%,174.60%,-0.54%
2,100.60%,44.40%,6.21%,-41.61%
3,-32.70%,-,-,-42.55%
4,-95.23%,-127.28%,-75.86%,42.65%


> Formatting techniques can be used in conjunction with styling:

In [14]:
df.style.highlight_max().format(None, na_rep="_")

,A,B,C,D
0,1.565390,_,_,0.505232
1,0.293497,1.412492,1.745990,-0.005426
2,1.005967,0.444021,0.062143,-0.416147
3,-0.326980,_,_,-0.425460
4,-0.952315,-1.272838,-0.758581,0.426546


# 4. Builtin Styles

In [15]:
df.style.highlight_null(null_color='orangered')

,A,B,C,D
0,1.565390,nan,nan,0.505232
1,0.293497,1.412492,1.745990,-0.005426
2,1.005967,0.444021,0.062143,-0.416147
3,-0.326980,nan,nan,-0.425460
4,-0.952315,-1.272838,-0.758581,0.426546


>You can create “heatmaps” with the `background_gradient` method

In [16]:
import matplotlib

df.style.background_gradient()  # requires matplotlib

,A,B,C,D
0,1.565390,nan,nan,0.505232
1,0.293497,1.412492,1.745990,-0.005426
2,1.005967,0.444021,0.062143,-0.416147
3,-0.326980,nan,nan,-0.425460
4,-0.952315,-1.272838,-0.758581,0.426546


In [17]:
df.style.highlight_max(subset=['B', 'D'], color='coral')

,A,B,C,D
0,1.565390,nan,nan,0.505232
1,0.293497,1.412492,1.745990,-0.005426
2,1.005967,0.444021,0.062143,-0.416147
3,-0.326980,nan,nan,-0.425460
4,-0.952315,-1.272838,-0.758581,0.426546


In [18]:
df.style.highlight_min(color='cyan')

,A,B,C,D
0,1.565390,nan,nan,0.505232
1,0.293497,1.412492,1.745990,-0.005426
2,1.005967,0.444021,0.062143,-0.416147
3,-0.326980,nan,nan,-0.425460
4,-0.952315,-1.272838,-0.758581,0.426546


In [19]:
df.style.hide_index()

A,B,C,D
1.565390,nan,nan,0.505232
0.293497,1.412492,1.745990,-0.005426
1.005967,0.444021,0.062143,-0.416147
-0.326980,nan,nan,-0.425460
-0.952315,-1.272838,-0.758581,0.426546


Use `Styler.set_properties` to apply  CSS styles arbitrarily

In [20]:
df.style.set_properties(**{'background-color': 'lime', 'font-style': 'oblique'})

,A,B,C,D
0,1.565390,nan,nan,0.505232
1,0.293497,1.412492,1.745990,-0.005426
2,1.005967,0.444021,0.062143,-0.416147
3,-0.326980,nan,nan,-0.425460
4,-0.952315,-1.272838,-0.758581,0.426546


In [21]:
df['E']=range(1, 6)
df.style.bar(subset=['A', 'C', 'E'], color='skyblue')

,A,B,C,D,E
0,1.565390,nan,nan,0.505232,1
1,0.293497,1.412492,1.745990,-0.005426,2
2,1.005967,0.444021,0.062143,-0.416147,3
3,-0.326980,nan,nan,-0.425460,4
4,-0.952315,-1.272838,-0.758581,0.426546,5


In [22]:
df.style.bar(subset=['A', 'B'], align='mid', color=['crimson', 'skyblue'])

,A,B,C,D,E
0,1.565390,nan,nan,0.505232,1
1,0.293497,1.412492,1.745990,-0.005426,2
2,1.005967,0.444021,0.062143,-0.416147,3
3,-0.326980,nan,nan,-0.425460,4
4,-0.952315,-1.272838,-0.758581,0.426546,5


# 5. Sharing Styles

>You can access an existing style using the `export` method, and apply it to a different dataset with the `use` method.

In [23]:
df_style1 = df.style.apply(enlarge_overall_max, axis=None)
df_style1

,A,B,C,D,E
0,1.565390,nan,nan,0.505232,1
1,0.293497,1.412492,1.745990,-0.005426,2
2,1.005967,0.444021,0.062143,-0.416147,3
3,-0.326980,nan,nan,-0.425460,4
4,-0.952315,-1.272838,-0.758581,0.426546,5


In [24]:
df2 = 1 / df**2
df2.style.use(df_style1.export())

,A,B,C,D,E
0,0.408089,nan,nan,3.917588,1.000000
1,11.608928,0.501219,0.328032,33971.723612,0.250000
2,0.988172,5.072152,258.948534,5.774403,0.111111
3,9.353128,nan,nan,5.524376,0.062500
4,1.102653,0.617240,1.737783,5.496277,0.040000


# 6. Options independent of the data

### 6.1 Precision

Setting the precision only affects the printed number.

In [25]:
df.style.set_precision(3)

,A,B,C,D,E
0,1.565,nan,nan,0.505,1
1,0.293,1.412,1.746,-0.005,2
2,1.006,0.444,0.062,-0.416,3
3,-0.327,nan,nan,-0.425,4
4,-0.952,-1.273,-0.759,0.427,5


### 6.2 Captions

In [26]:
df.style.set_caption('Some useful caption here')

,A,B,C,D,E
0,1.565390,nan,nan,0.505232,1
1,0.293497,1.412492,1.745990,-0.005426,2
2,1.005967,0.444021,0.062143,-0.416147,3
3,-0.326980,nan,nan,-0.425460,4
4,-0.952315,-1.272838,-0.758581,0.426546,5


### 6.3 Table Styles

`table_styles` should be a list of dictionaries, each having the `selector` and `props` keys. 

`selector` should be a valid CSS selector. `props` should be a list of tuples of ('attribute', 'value').

In [27]:
from IPython.display import HTML

def hover(hover_color="yellow"):
    return {'selector': "tr:hover",
            'props': [("background-color", f"{hover_color}")]}

styles = [
    hover(),
    {'selector': "th", 'props': [("font-size", "150%"),
                                 ("text-align", "center"),
                                 ("color", "slategrey")]},
    {'selector': "caption", 'props': [("caption-side", "bottom"),
                                      ("font-style", "italic")]}
]
html = (df.style.set_table_styles(styles)
          .set_caption("Hover to highlight."))
html

,A,B,C,D,E
0,1.565390,nan,nan,0.505232,1
1,0.293497,1.412492,1.745990,-0.005426,2
2,1.005967,0.444021,0.062143,-0.416147,3
3,-0.326980,nan,nan,-0.425460,4
4,-0.952315,-1.272838,-0.758581,0.426546,5


### 6.4 Missing Values

In [28]:
df.style.set_na_rep('\ | /').highlight_null("yellow")

,A,B,C,D,E
0,1.565390,\ | /,\ | /,0.505232,1
1,0.293497,1.412492,1.745990,-0.005426,2
2,1.005967,0.444021,0.062143,-0.416147,3
3,-0.326980,\ | /,\ | /,-0.425460,4
4,-0.952315,-1.272838,-0.758581,0.426546,5


### 6.5 Hiding the Index or Columns

In [29]:
df.style.hide_index()

A,B,C,D,E
1.565390,nan,nan,0.505232,1
0.293497,1.412492,1.745990,-0.005426,2
1.005967,0.444021,0.062143,-0.416147,3
-0.326980,nan,nan,-0.425460,4
-0.952315,-1.272838,-0.758581,0.426546,5


In [30]:
df.style.hide_columns(['B','E'])

,A,C,D
0,1.565390,nan,0.505232
1,0.293497,1.745990,-0.005426
2,1.005967,0.062143,-0.416147
3,-0.326980,nan,-0.425460
4,-0.952315,-0.758581,0.426546


# 7. Interesting Features ("Fun Stuff")

In [31]:
from IPython.html import widgets

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "4pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

df = pd.DataFrame(np.random.rand(25, 25))
df.style.background_gradient( axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '1pt'})\
    .set_caption("Hover to magnify")\
    .set_precision(2)\
    .set_table_styles(magnify())

/home/tim/Projects/learning-pandas/venv/lib/python3.8/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,0.86,0.63,0.82,0.95,0.61,0.26,0.17,0.91,0.04,0.29,0.14,0.17,0.57,0.28,0.75,0.98,0.26,0.71,0.79,0.38,0.95,0.19,0.22,0.91,0.51
1,0.43,0.38,0.04,0.99,0.14,0.21,0.46,0.26,0.48,0.87,0.84,0.32,0.69,0.96,0.91,0.09,0.98,0.66,0.24,0.05,0.32,0.01,0.65,0.71,0.03
2,0.25,0.28,0.36,0.46,0.71,0.14,0.10,0.32,0.87,0.06,0.13,0.14,0.71,0.78,0.51,0.72,0.77,0.59,0.56,0.90,0.49,0.18,0.77,0.09,0.70
3,0.39,0.44,0.73,0.95,0.90,0.69,0.13,0.01,0.31,0.83,0.28,0.47,0.30,0.45,0.89,0.94,0.94,0.53,0.45,0.52,0.50,0.88,0.96,0.08,0.34
4,0.81,0.82,0.30,0.47,0.46,0.87,0.68,0.80,0.68,0.30,0.25,0.34,0.55,0.27,0.11,0.82,0.85,0.12,0.22,0.24,0.60,0.62,0.77,0.44,0.59
5,0.21,1.00,0.94,0.83,0.17,0.62,0.51,0.20,0.08,0.94,0.08,0.06,0.40,0.12,0.52,0.79,0.36,0.91,0.36,0.97,0.58,0.17,0.25,0.48,0.95
6,0.92,0.06,0.93,0.18,0.17,0.07,0.55,0.19,0.02,0.82,0.41,0.24,0.20,0.70,0.27,0.26,0.16,0.50,0.01,0.07,0.67,0.64,0.46,0.02,0.60
7,0.59,0.79,0.96,0.99,0.34,0.32,0.23,0.19,0.80,0.33,0.88,0.71,0.04,0.88,0.84,0.21,0.85,0.38,0.35,0.85,0.00,0.09,0.34,0.31,0.20
8,0.04,0.89,0.74,0.27,0.27,0.34,0.24,0.23,0.49,0.74,0.54,0.40,0.17,0.17,0.30,0.74,0.06,0.41,0.96,0.33,0.97,0.41,0.76,0.06,0.33
9,0.87,0.08,0.35,0.34,0.66,0.38,0.63,0.20,0.69,0.99,0.78,0.69,0.58,0.40,1.00,0.26,0.16,0.58,0.72,0.61,0.72,0.40,0.87,0.54,0.24


# 8. Exporting to Excel

- This feature is still experimenatal and subject to changes/enhancements.
- Some support is available for exporting styled `DataFrames` to Excel worksheets using the `OpenPyXL` or `XlsxWriter` engines.
- Only CSS2 named colors and hex colors of the form `#rgb` or `#rrggbb` are currently supported. Properties available include:
```
    background-color       white-space: nowrap

    border-style, border-width, border-color and their {top, right, bottom, left variants}

    color      font-family      font-style     font-weight

    text-align       text-decoration       vertical-align
```

In [32]:
df.style.apply(highlight_max)\
    .to_excel('styled.xlsx', engine='openpyxl')